## ДЗ по поиску

Привет! Вам надо реализивать поисковик на базе вопросов-ответов с сайта [pravoved.ru](https://pravoved.ru/questions-archive/).        
Поиск должен работать на трех технологиях:       
1. обратном индексе     
2. word2vec         
3. doc2vec      

Вы должны понять, какой метод и при каких условиях эксперимента на этом корпусе работает лучше.          
Для измерения качества поиска найдите точность (accuracy) выпадания правильного ответа на конкретный вопрос (в этой базе у каждого вопроса есть только один правильный ответ). Точность нужно измерить для всей базы.    
При этом давайте считать, что выпал правильный ответ, если он попал в **топ-5** поисковой выдачи.

> Сделайте ваш поиск максимально качественным, чтобы значение точности стремилось к 1.     
Для этого можно поэкспериментировать со следующим:       
- модель word2vec (можно брать любую из опен сорса или обучить свою)
- способ получения вектора документа через word2vec: простое среднее арифметическое или взвешивать каждый вектор в соответствии с его tf-idf      
- количество эпох у doc2vec (начинайте от 100)
- предобработка документов для обучения doc2vec (удалять / не удалять стоп-слова)
- блендинг методов поиска: соединить результаты обратного индекса и w2v, или (что проще) w2v и d2v

На это задание отведем 10 дней. Дэдлайн сдачи до полуночи 12.10.

In [1]:
import pickle

with open('qa_corpus.pkl', 'rb') as file:
    qa_corpus = pickle.load(file)

Всего в корпусе 1384 пары вопрос-ответ

In [21]:
type(qa_corpus)

list

Первый элемент блока это вопрос, второй - ответ на него

In [7]:
qa_corpus[1][0]

'\nЗдравствуйте, я продаю комнату в 4х комнатной квартире. С одним из соседей у меня долевка. Лицевые счета разные. Риелтор говорит что при продаже нужно будет оплачивать натариусу налог с дохода. 17400. Продала за 450000. Сама буду брать квартиру в ипотеку.\n'

## Word2vec

#### Начнем с модели из опенсорса без pos-tagging

In [2]:
import warnings
warnings.filterwarnings(action='ignore', module='gensim')

import gensim
from gensim.models import Word2Vec, KeyedVectors

In [3]:
from tqdm import tqdm_notebook as tqdm
import re
from judicial_splitter import splitter
import pandas as pd

In [9]:
model_path = 'araneum_none_fasttextcbow_300_5_2018/araneum_none_fasttextcbow_300_5_2018.model'
model = Word2Vec.load(model_path)

In [4]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string


def preprocessing(input_text, del_stopwords=True, del_digit=True):
    """
    :input: raw text
        1. lowercase, del punctuation, tokenize
        2. normal form
        3. del stopwords
        4. del digits
    :return: lemmas
    """
    russian_stopwords = set(stopwords.words('russian'))
    words = [x.lower().strip(string.punctuation+'»«–…') for x in word_tokenize(input_text)]
    lemmas = [morph.parse(x)[0].normal_form for x in words if x]

    lemmas_arr = []
    for lemma in lemmas:
        if del_stopwords:
            if lemma in russian_stopwords:
                continue
        if del_digit:
            if lemma.isdigit():
                continue
        lemmas_arr.append(lemma)
    return lemmas_arr

In [5]:
def no_spaces(text):
    
    processed_text = text.replace('\n', ' ').replace('\n\n', ' ').replace('ул. ', 'ул.').replace('г. ', 'г.').replace('гор. ', 'гор.').replace('с. ', 'с.')
    return processed_text


# убираем пробел после инициалов перед фамилией
def clear_abbrs(processed_text):
    initials = re.compile(r'[А-Я]{1}\.[А-Я]{1}\. [А-Я][а-яё]+')
    counter = len(initials.findall(processed_text))

    for s in range(counter):
        get_abbrs = initials.search(processed_text)
        i = get_abbrs.span()[0] + 4
        processed_text = processed_text[:i] + processed_text[i+1:]
    return processed_text

In [7]:
# this function works with preprocessing func result
def get_w2v_vectors(lemmas_arr, model):
    """Получает вектор документа"""
    if len(lemmas_arr) == 0:
        doc_vec = None
    else:
        vectors = []
        for element in lemmas_arr:
            try:
                vec = model.wv[element]
                # len(vec) this gives us 300
            except KeyError:
                continue
            vectors.append(vec)
        vec_sum = np.zeros(300)
        for v in vectors:
            vec_sum += v
        doc_vec = vec_sum/len(vectors)   # усредненный вектор как опознаватель
    return doc_vec

In [8]:
# индексирует вопросы
def save_w2v_base(qa_corpus, model):
    """Индексирует всю базу для поиска через word2vec"""
    col_names =  ['question_vec', 'question', 'answer']
    dc  = pd.DataFrame(columns = col_names)
    for pair in tqdm(qa_corpus):
        text, answer = pair[0], pair[1]
        text = no_spaces(text)
        question = clear_abbrs(text)
        vec = get_w2v_vectors(preprocessing(question), model)
        dc.loc[len(dc)] = [vec, question, answer]
    return dc

In [47]:
dc = save_w2v_base(qa_corpus, model)

In [48]:
dc.head()

,question_vec,question,answer
0,"[0.020457512601730003, 0.006679873259383298, -...","Добрый день.Мой сын гражданин Украины (ДНР),и...",Добрый вечер!Из Вашего вопроса вообще ничего н...
1,"[-0.00012631115058194035, 0.004088216117056815...","Здравствуйте, я продаю комнату в 4х комнатной...","Оксана, Вы вправе не платить налог, если являе..."
2,"[0.007701517820047836, -0.015528799515474626, ...",Можно ли подать приложения к жалобе по делу о...,"Здравствуйте, Илья! Можно ли подать приложения..."
3,"[-0.004688120456800486, 0.00814481066643364, 0...",Добрый вечер.\r Произошла мало приятная ситуа...,Представьте органу предварительного расследова...
4,"[-0.017411477700807153, 0.007722027279669419, ...",Я брала займ 5000 т.р произошли трудности в ж...,"Прямо в полицию и обращайтесь, в заявлении про..."


In [9]:
from gensim import matutils

def similarity(v1, v2):
    v1_norm = matutils.unitvec(np.array(v1))
    v2_norm = matutils.unitvec(np.array(v2))
    return np.dot(v1_norm, v2_norm)

In [10]:
def search_w2v(query, model_vectors, doc_text, model):
    """
    :doc_text: questions?
    :query: all answers
    """
    lemmas_query = preprocessing(query)
    query_vector = get_w2v_vectors(lemmas_query, model)
    # print(type(query_vector))
    result = []
    i = 0
    for vec in model_vectors:
        # print(type(vec))
        similar = similarity(query_vector, vec)
        result.append((doc_text[i], similar))
        i += 1
    res = pd.DataFrame(result, columns=['question','similarity'])
    return res.sort_values('similarity', ascending=False)  # sort by similarity

In [75]:
# test run for accuracy
correct = 0
for i, answer in tqdm(enumerate(dc['answer'])):
    res = search_w2v(answer, dc['question_vec'], dc['question'], model)[:5]['question']
    if dc['question'][i] in res:
        correct += 1
print(correct)


0


Accuracy = 0 :( Ничего ни разу не совпало. Ну значит, это плохой метод.

In [17]:
def accuracy(all_answers, question_vec, all_questions, model):
    correct = 0
    for i, answer in tqdm(enumerate(all_answers)):
        res = search_w2v(answer, question_vec, all_questions, model)[:5]['question']
        if all_questions[i] in res:
            correct += 1
    return correct

Можно разбить выборку и сравнивать вопросы с вопросами, а не ответы с вопросами. Например, 30% всего корпуса будут вопросы как бы от пользователя, а база, по которой ищем будет состоять из 70% корпуса. И тогда мы будем сравнивать вопросы пользователей с вопросами из базы. Но тогда мы не сможем посчитать accuracy, потому что правильный ответы будут как бы неизвестны.

Еще можно обучить свою модель word2vec, например, на части текстов из нашего корпуса, например на 80 %, а потом потестить эту модель на 20 %.

In [13]:
documents = []
for el in tqdm(qa_corpus[:1107]):  # first 80%
    q = el[0]
    q = preprocessing(q)
    documents.append(q)
    
my_model = gensim.models.Word2Vec(
        documents,
        size=300,
        window=10,
        min_count=2,
        workers=4)
my_model.train(documents, total_examples=len(documents), epochs=30)    

(1056400, 1273800)

In [14]:
dc2 = save_w2v_base(qa_corpus[1107:], my_model)

In [15]:
dc2.head()

,question_vec,question,answer
0,"[0.12066721859829206, 0.010513915582314917, -0...",добрый день с наступающим новым годом работал...,Здравствуйте. Можете обратиться в инспекцию по...
1,"[0.12527427914652176, 0.06167132819869688, -0....",Добрый день. Попал в аварию вызвали комиссара...,"Михаил, вам необходимо было второй экземпляр е..."
2,"[-0.10231609290457089, -0.09004459154136756, -...",сдраствуйте у меня такая ситуацыя я щас нахож...,"Александр,как правило, в таких случаях наказан..."
3,"[-0.06411619518573085, -0.08648401313306144, -...",До какого времени можно играть на музыкальных...,В каждом субъекте федерации имеется свои норма...
4,"[0.048908219197574924, -0.02549409524214511, -...",Здравствуйте.у меня такой вопрос я взял ипоте...,"Добрый вечер, Василий!Первичный документ для н..."


In [18]:
correct = accuracy(dc2['answer'], dc2['question_vec'], dc2['question'], my_model)
print(correct)


0


## Doc2vec

## Inverted Index

## Блендинг методов поиска